In [ ]:
import os
import random
import shutil
import imgaug.augmenters as iaa
import cv2
from imgaug.augmentables.batches import UnnormalizedBatch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/deep_learning')

In [ ]:
train_folder = os.path.join("", "train")
val_folder = os.path.join("", "val")
test_folder = os.path.join("", "test")
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

In [ ]:
seq = iaa.Sequential([
    iaa.Affine(rotate=(-90, 90)),  # Rotación aleatoria entre -20 y 20 grados
    iaa.Fliplr(0.5),  # Volteo horizontal aleatorio (50% de probabilidad)
    # iaa.GaussianBlur(sigma=(0, 0.1)),  # Desenfoque gaussiano con sigma aleatorio entre 0 y 1.0
    # iaa.AdditiveGaussianNoise(scale=(0, 0.05 * 255)),  # Ruido gaussiano con escala aleatoria
    iaa.Multiply((0.6, 1.2))  # Multiplicación de brillo aleatoria
])

In [ ]:
melanoma_folder = os.path.join("", "melanoma-augmented")
file_list = os.listdir(melanoma_folder)
print(len(file_list))

6678


In [ ]:
for filename in file_list:
    # Comprobar si el archivo contiene "augmented" en su nombre
    if "augmented" in filename:
        print('removed')
        # Construir la ruta completa del archivo
        file_path = os.path.join(melanoma_folder, filename)
        
        # Eliminar el archivo
        os.remove(file_path)

In [ ]:
image_path = os.path.join(melanoma_folder, file_list[0])
image = cv2.imread(image_path)
print(file_list[0])

# Generar imágenes aumentadas
for i in range(5):
  # Aplicar las transformaciones de data augmentation
  augmented_image = seq.augment_image(image)
  
  # Construir el nombre de archivo de salida
  output_filename = f"{file_list[0].split('.')[0]}_augmented_{i}.{file_list[0].split('.')[-1]}"
  print(output_filename)
  output_path = os.path.join(melanoma_folder, output_filename)
  
  # Guardar la imagen aumentada en la carpeta de salida
  cv2.imwrite(output_path, augmented_image)

ISIC_0025235.jpg
ISIC_0025235_augmented_0.jpg
ISIC_0025235_augmented_1.jpg
ISIC_0025235_augmented_2.jpg
ISIC_0025235_augmented_3.jpg
ISIC_0025235_augmented_4.jpg


In [ ]:
for filename in file_list:
    # Comprobar si el archivo es una imagen (por extensión)
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Cargar la imagen original
        image_path = os.path.join(melanoma_folder, filename)
        image = cv2.imread(image_path)
        
        # Aplicar data augmentation a la imagen original
        for i in range(3):
          # Aplicar las transformaciones de data augmentation
          augmented_image = seq.augment_image(image)
          
          # Construir el nombre de archivo de salida
          output_filename = f"{filename.split('.')[0]}_augmented_{i}.{filename.split('.')[-1]}"
          output_path = os.path.join(melanoma_folder, output_filename)
          
          # Guardar la imagen aumentada en la carpeta de salida
          cv2.imwrite(output_path, augmented_image)

In [ ]:
melanoma_folder = os.path.join("", "melanoma")
melanoma_images = [f for f in os.listdir(melanoma_folder) if os.path.isfile(os.path.join(melanoma_folder, f))]

In [ ]:
print(len(melanoma_images))

1113


In [ ]:
non_melanoma_folder = os.path.join("", "reduced-non-melanoma")
non_melanoma_images = [f for f in os.listdir(non_melanoma_folder) if os.path.isfile(os.path.join(non_melanoma_folder, f))]

In [ ]:
print(len(non_melanoma_images))

1272


In [ ]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

In [ ]:
num_melanoma = len(melanoma_images)
num_non_melanoma = len(non_melanoma_images)
num_train_melanoma = int(num_melanoma * train_ratio)
num_train_non_melanoma = int(num_non_melanoma * train_ratio)
num_val_melanoma = int(num_melanoma * val_ratio)
num_val_non_melanoma = int(num_non_melanoma * val_ratio)
num_test_melanoma = num_melanoma - num_train_melanoma - num_val_melanoma
num_test_non_melanoma = num_non_melanoma - num_train_non_melanoma - num_val_non_melanoma

In [ ]:
random.shuffle(melanoma_images)
train_melanoma = melanoma_images[:num_train_melanoma]
val_melanoma = melanoma_images[num_train_melanoma:num_train_melanoma+num_val_melanoma]
test_melanoma = melanoma_images[num_train_melanoma+num_val_melanoma:]

In [ ]:
random.shuffle(non_melanoma_images)
train_non_melanoma = non_melanoma_images[:num_train_non_melanoma]
val_non_melanoma = non_melanoma_images[num_train_non_melanoma:num_train_non_melanoma+num_val_non_melanoma]
test_non_melanoma = non_melanoma_images[num_train_non_melanoma+num_val_non_melanoma:]

In [ ]:
for image in train_melanoma:
    src = os.path.join(melanoma_folder, image)
    dst = os.path.join(train_folder, "melanoma", image)
    shutil.copy(src, dst)

for image in train_non_melanoma:
    src = os.path.join(non_melanoma_folder, image)
    dst = os.path.join(train_folder, "non-melanoma", image)
    shutil.copy(src, dst)

for image in val_melanoma:
    src = os.path.join(melanoma_folder, image)
    dst = os.path.join(val_folder, "melanoma", image)
    shutil.copy(src, dst)

for image in val_non_melanoma:
    src = os.path.join(non_melanoma_folder, image)
    dst = os.path.join(val_folder, "non-melanoma", image)
    shutil.copy(src, dst)

for image in test_melanoma:
    src = os.path.join(melanoma_folder, image)
    dst = os.path.join(test_folder, "melanoma", image)
    shutil.copy(src, dst)

for image in test_non_melanoma:
    src = os.path.join(non_melanoma_folder, image)
    dst = os.path.join(test_folder, "non-melanoma", image)
    shutil.copy(src, dst)